# Advanced Spatial Analysis: Spatial Autocorrelation

## Overview
In this lecture, we will learn spatial autocorrelation with two well-known methods: Moran's I and Local Indicator of Spatial Association (LISA). 
* **Global Moran's I** demonstrates how geographical phenomena are correlated over space, meaning whether closer things is more related than distant things. The method provides an index with the range -1 to 1; namely, -1 is a strong negative spatial autocorrelation and 1 is a strong positive spatial autocorrelation. 
* While Global Moran's I only provides one index to demonstrate spatial autocorrelation, **Local Indicator of Spatial Association (LISA)**, as known as Local Moran's I explains where high (i.e., HH Cluster) and low (LL Cluster) values are clustered. 

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import esda
import libpysal
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

## Spatial Autocorrelation: Global Moran's I

Spatial autocorrelation investigates how the geographical phenomena **are spatially related** to each other based on Tober's First Law of Geography; meaning that everything is usually related to all else but those which are near to each other are more related when compared to those that are further away. There are several indices that indicate the degree of spatial autocorrelation (e.g., Geary's C or Getis-Ord Gi*). Here, we study Moran's I, which is the most well-known method. 

$$I = \frac{n}{W} \frac{\sum_{i=1}^{n}\sum_{j=1}^{n} w_{ij}(x_i - \overline{x})(x_j - \overline{x})}{\sum_{i=1}^{n}(x_i - \overline{x})^2}$$

where <br>
$n$ - the number of objects in space, <br>
$W$ - the sum of spatial weights, <br>
$w$ - a spatial weight for a pair of objects, <br>
$x_i, x_j$ - values of an attribute for objects i and j, <br>
$\overline{x}$ - a mean value of an attribute. <br>




In [ ]:
# Census tract shapefile of Illinois
tract = gpd.read_file('./data/tl_2020_17_tract.shp')
tract = tract[['GEOID', 'geometry']]
tract = tract.loc[~tract['GEOID'].isin(['17031990000', '17097990000'])]
tract.head()

In [ ]:
# ACS Household income data
income_table = pd.read_csv('./data/household_income.csv')
income_table['GEOID'] = income_table['GEOID'].astype(str)
income_table.head()

In [ ]:
# Join two dataframes 
income = tract.merge(income_table, on='GEOID')
income['MeanIncome'] = income['MeanIncome'].replace('-', '0') # Replace missing value with 0
income['MeanIncome'] = income['MeanIncome'].replace('N', '0') # Replace missing value with 0
income['MeanIncome'] = income['MeanIncome'].astype(int)
income.head()

In [ ]:
# Select only Cook County
cook_income = income.loc[income['GEOID'].str.startswith('17031')]
cook_income = cook_income.to_crs(epsg=26971)
cook_income = cook_income.reset_index(drop=True)
cook_income

In [ ]:
# Geographical distribution of Houshold Mean Income data
ax = cook_income.plot('MeanIncome', scheme='FisherJenks', k=7, cmap='Blues', figsize=(15, 10), legend=True)

leg = ax.get_legend()
leg.set_bbox_to_anchor((0.4, 0.25))

### Backbone of calculating Moran's I with Python

```python
import libpysal
import esda

## 1. Calculate weights (w) of geographical units
w = libpysal.weights.Queen() # based on Rook's case contiguity
w = libpysal.weights.Rook()  # based on Queen's case contiguity
w = libpysal.weights.DistanceBand() # based on a fixed distance

## 2. Define value to calculate spatial autocorrelation
y = df['MeanIncome']

## 3. Calculate the final index
mi = esda.moran.Moran(y, w_queen)
print(mi.I) # Moran's I value
print(mi.p_norm) # p-value of the current Moran's I 
print(mi.z_norm) # Z Score of the current Moran's I 

```

### Compute weigts of a geographical unit over the other geographical units
#### Based on contiguity: Rook's case and Queen's case

For more information, visit <a href=https://pysal.org/libpysal/generated/libpysal.weights.Queen.html>libpysal.weights.Queen</a> or <a href=https://pysal.org/libpysal/generated/libpysal.weights.Rook.html>libpysal.weights.Rook</a>.

In [ ]:
w_queen = libpysal.weights.Queen.from_dataframe(cook_income)
w_rook = libpysal.weights.Rook.from_dataframe(cook_income)

fig, axes = plt.subplots(1, 2, figsize=(20,10))

# Contiguity: Rook's case
cook_income.boundary.plot(ax=axes[0], ls=':', color='black')
w_rook.plot(cook_income, ax=axes[0], 
            edge_kws=dict(color='r', linestyle=':', linewidth=1),
            node_kws=dict(marker=''))

# Contiguity: Queen's case
cook_income.boundary.plot(ax=axes[1], ls=':', color='black')
w_queen.plot(cook_income, ax=axes[1], 
             edge_kws=dict(color='r', linestyle=':', linewidth=1),
             node_kws=dict(marker=''))

You can use `.neighbors` attribute to check the neighbor of each geographical unit and `.weights` attribute to check their weights. For contiguity, weights are automatically assigned to equal values. 

In [ ]:
w_rook.neighbors

In [ ]:
w_rook.weights

Based on contiguity criteria, the list of neighbors is different.

In [ ]:
w_rook.neighbors[1]

In [ ]:
w_queen.neighbors[1]

#### Calculate Moran's I

To calculate Moran's I, you can simply enter the attribute of interest (i.e., `MeanIncome`) and weight matrix (i.e., `w_rook` or `w_queen`) to <a href=https://pysal.org/esda/generated/esda.Moran.html>esda.moran.Moran()</a>.

In [ ]:
y = cook_income['MeanIncome']

mi_rook = esda.moran.Moran(y, w_rook)
mi_queen = esda.moran.Moran(y, w_queen)
print(f"Moran's I with Rook's case contiguity: {round(mi_rook.I, 3)}, p-value: {round(mi_rook.p_norm, 3)}")
print(f"Moran's I with Queen's case contiguity: {round(mi_queen.I, 3)}, p-value: {round(mi_queen.p_norm, 3)}")

### Use of Fixed distance to calculate neighbors

The drawback of contiguity based neighbors is that they do not consider the distance decay for calculating weights. Here, we examine a way to incorporate a distance decay functions. 

The distance decay function in this package is as shown below. Here, alpha value should be negative. If the alpha value decreases, the distance decay becomes strong.

$$w_{ij} = d_{ij}^\alpha$$

In [ ]:
# The effect of the power on distance decay
x = np.linspace(1,10,10)
y_05 = [val**-0.5 for val in x]
y_1 = [val**-1 for val in x]
y_15 = [val**-1.5 for val in x]

plt.plot(x, y_05, label='alpha = -0.5')
plt.plot(x, y_1, label='alpha = -1')
plt.plot(x, y_15, label='alpha = -1.5')

plt.legend(fontsize=15)
plt.show()

In [ ]:
cook_income

In [ ]:
dist = 1000 # distance band
alpha_val = -1 # the power of distance decay function, should be negative value. 

# Obtain coordinates of each geographical units
coords = cook_income.apply(lambda x:x.geometry.centroid.coords[0], axis=1).values

# Calculate weights of each geographical units based on distance decay method. 
w = libpysal.weights.DistanceBand(data=list(coords), # coordinates of each geographical units
                                  threshold=dist, # distance band
                                  binary=False, # whether distance decay is employed or not. 
                                  alpha=alpha_val, # distance decay parameter for weight (default -1.0)
                                  silence_warnings=True
                                 )
print(w.weights[3])
print(w.neighbors[3])

In [ ]:
w.neighbors

In [ ]:
print(w_rook.neighbors[3])
print(w_queen.neighbors[3])

In [ ]:
for i in w.neighbors[3]:
    print(i, cook_income.at[3, 'geometry'].centroid.distance(cook_income.at[i, 'geometry'].centroid))

In [ ]:
m = cook_income.loc[cook_income.index == 3].explore(color='black')
cook_income.loc[cook_income.index.isin(w.neighbors[3])].explore(m=m, color='blue')

In [ ]:
temp_dist = cook_income.at[3, 'geometry'].centroid.distance(cook_income.at[15, 'geometry'].centroid)
print(f'Distance: {round(temp_dist)} meters, weight per distance decay: {temp_dist ** -1}')

In [ ]:
# Calculate Moran's I
mi = esda.moran.Moran(y, w)

# Print results
print(mi.I) # Moran's I value
print(mi.p_norm) # p-value of the current Moran's I 
print(mi.z_norm) # Z Score of the current Moran's I 

print(f"Moran's I with {dist} meter radius: {round(mi.I, 3)}, p-value: {round(mi.p_norm, 3)}, z-score: {round(mi.z_norm, 3)}")

In [ ]:
# Plot relationship between geographical units with a given distance band
fig, ax = plt.subplots(figsize=(10, 10))
cook_income.boundary.plot(ax=ax, ls=':', color='black')
w.plot(cook_income, ax=ax, 
       edge_kws=dict(color='blue', linestyle=':', linewidth=1),
       node_kws=dict(marker=''))

plt.show()

---
### *Exercise*

Here you will examine the sensitivity of Moran's I with **various threshold bandwidths and various alpha for distance decay**. Also, think about which combination of the weight of distance decay and threshold bandwidth provides the most statistically significant result. You can check <a href=https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/choosingdistanceband.htm>this website</a> for more information. <br>

The following codes are the backbone of spatial autocorrelation analysis. 

```python
# Your code here
dist = 3000 # threshold bandwidth
alpha_val = -1 # the power of distance decay function, should be negative value. 

# Obtain coordinates of each geographical units
coords = cook_income.apply(lambda x:x.geometry.centroid.coords[0], axis=1).values

# Calculate weights of each geographical units based on distance decay method. 
w = libpysal.weights.DistanceBand(data=list(coords), # coordinates of each geographical units
                                  threshold=dist, # threshold bandwidth
                                  binary=False, # whether distance decay is employed or not. 
                                  alpha=alpha_val, # distance decay parameter for weight (default -1.0)
                                  silence_warnings=True
                                 )
# Focused variable
y = cook_income['MeanIncome']

# Calculate Moran's I
mi = esda.moran.Moran(y, w)

# Print results
print(f"Moran's I with {dist} meter radius: {round(mi.I, 3)}, p-value: {round(mi.p_norm, 3)}, z-score: {round(mi.z_norm, 3)}")
```

---

In [ ]:
# Your code here
dist = 3000 # threshold bandwidth
alpha_val = -1 # the power of distance decay function, should be negative value. 

# Obtain coordinates of each geographical units
coords = cook_income.apply(lambda x:x.geometry.centroid.coords[0], axis=1).values

# Calculate weights of each geographical units based on distance decay method. 
w = libpysal.weights.DistanceBand(data=list(coords), # coordinates of each geographical units
                                  threshold=dist, # threshold bandwidth
                                  binary=False, # whether distance decay is employed or not. 
                                  alpha=alpha_val, # distance decay parameter for weight (default -1.0)
                                  silence_warnings=True
                                 )
# Focused variable
y = cook_income['MeanIncome']

# Calculate Moran's I
mi = esda.moran.Moran(y, w)

# Print results
print(f"Moran's I with {dist} meter radius: {round(mi.I, 3)}, p-value: {round(mi.p_norm, 3)}, z-score: {round(mi.z_norm, 3)}")

## Local Indicators of Spatial Association (LISA): Local Moran's I

Moran's I is a characteristic of the complete spatial pattern and does not provide an indication of the location of the clusters. The concept of a local indicator of spatial association, or LISA was suggested in Anselin (1995) to remedie this situation. A LISA is seen as having two important characteristics. 
1. It provides a statistic for each location with an assessment of significance. 
2. It establishes a proportional relationship between the sum of the local statistics and a corresponding global statistic.

Source: https://geodacenter.github.io/workbook/6a_local_auto/lab6a.html <br>
Anselin, Luc. 1995. “Local Indicators of Spatial Association — LISA.” Geographical Analysis 27: 93–115.

<a href=https://pysal.org/esda/generated/esda.Moran_Local.html>esda.moran.Moran_Local</a> will help you calculate LISA. It returns two important information in `.q` attribute and `p_sim`. `.q` provides the indicator of each classification of LISA (1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'), and `p_sim` provide the p-value of each classification. 

In [ ]:
# https://pysal.org/esda/generated/esda.Moran_Local.html
y = cook_income['MeanIncome']  # Focused Variable
w_rook = libpysal.weights.Rook.from_dataframe(cook_income) # Contiguity weight

lm_rook = esda.moran.Moran_Local(y, w_rook)
print(lm_rook.q) # Classification of LISA
print(lm_rook.p_sim) # Significance of each classification

In [ ]:
lm_dict = {1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'}

lisa_rook = []
for idx in range(len(lm_rook.q)):
    if lm_rook.p_sim[idx] < 0.05:
        lisa_rook.append(lm_dict[lm_rook.q[idx]])
    else:
        lisa_rook.append('Not_Sig')
        
lisa_rook = pd.Series(lisa_rook)
cook_income['rook'] = lisa_rook
cook_income

In [ ]:
lisa_color = {'HH': 'red', 'LL': 'blue', 'HL': 'orange', 'LH': 'skyblue', 'Not_Sig': 'lightgrey'}

fig, ax = plt.subplots(figsize=(10, 10))

for key in lisa_color.keys():
    cook_income.loc[cook_income['rook'] == key].plot(ax=ax, color=lisa_color[key], legend=True)

plt.show()

---
### *Exercise*

Let's investigate how the local indicators of spatial association (LISA) varies with different weight (i.e., queen's case contiguity and the fixed bandwidth of 10000). Utilize the codes mentioned above and create two maps of LISA. The following describes the steps you need to do.

* LISA with Queen's case contiguity 
1. Create contiguity with `libpysal.weights.Queens.from_dataframe()`.
2. Run `esda.moran.Moran_Local()` to obtain LISA with `.q` and `.p_sim` attribute. 
3. Select label (i.e., `.q`) with a certain significance in `.p_sim` attribute.
4. Display the result

* Lisa with Fixed Band Width (10000 feet)
1. Extract points coordinates from the GeoDataFrame.
2. Calculate weight with `libpysal.weights.DistanceBand()` method. 
3. Run `esda.moran.Moran_Local()` to obtain LISA with `.q` and `.p_sim` attribute. 
4. Select label (i.e., `.q`) with a certain significance in `.p_sim` attribute.
5. Display the result

Check out the following websites for more information.
* https://pysal.org/esda/generated/esda.Moran_Local.html
* https://pysal.org/libpysal/generated/libpysal.weights.Queen.html
* https://pysal.org/libpysal/generated/libpysal.weights.DistanceBand.html


In [ ]:
# Your code here with Queen's case





In [ ]:
# Your code here with Fixed bandwidth





In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for key in lisa_color.keys():
    cook_income.loc[cook_income['rook'] == key].plot(ax=axes[0], color=lisa_color[key], legend=True)
    cook_income.loc[cook_income['queen'] == key].plot(ax=axes[1], color=lisa_color[key], legend=True)
    cook_income.loc[cook_income['fixed'] == key].plot(ax=axes[2], color=lisa_color[key], legend=True)


for ax in axes:
    ax.get_xaxis().set_visible(False)  # Remove ticks and labels
    ax.get_yaxis().set_visible(False)  # Remove ticks and labels
    
axes[0].set_title("Rook's case", fontsize=15)
axes[1].set_title("Queen's case", fontsize=15)
axes[2].set_title("Fixed bandwidth", fontsize=15)

plt.show()

## Challenge: manually write codes for Moran's I

Here we want to challenge ourselves to write codes for calculating Moran's I. As mentioned earlier, the equation looks like the one below. 

$$I = \frac{n}{W} \frac{\sum_{i=1}^{n}\sum_{j=1}^{n} w_{ij}(x_i - \overline{x})(x_j - \overline{x})}{\sum_{i=1}^{n}(x_i - \overline{x})^2}$$

where <br>
$n$ - the number of objects in space, <br>
$W$ - the sum of spatial weights, <br>
$w$ - a spatial weight for a pair of objects, <br>
$x_i, x_j$ - values of an attribute for objects i and j, <br>
$\overline{x}$ - a mean value of an attribute. <br>

It may sound very challenging, but you can achieve it by following the steps below. 
1. $\overline{x}$: Calculate the mean of the variable interested (Median income in our case). <br><br>
2. $(x_i - \overline{x})$: Calculate for each object a difference between single value and a mean. <br><br>
3. ${\sum_{i=1}^{n}(x_i - \overline{x})^2}$: Square each difference calculated at the previous step and to get a sum of these squares. <br><br>
4. $w_{ij} = d_{ij}^\alpha$: Calculate the distance decay of the pair locations based on the power of -1. <br><br>
5. $W = \sum_{i=1}^{n}\sum_{j=1}^{n} w_{ij}$: Sum the spatial weights. <br><br>
6. $\frac{n}{W}$: Simply divide the number of objects ($n$) by the sum of spatial weights ($W$). <br><br>
7. $\sum_{i=1}^{n}\sum_{j=1}^{n} w_{ij}(x_i - \overline{x})(x_j - \overline{x})$: Finalize the numerator <br><br>
8. Combine all functions together. 


### Step 1 — 3

In [ ]:
# Step 1
mean_score = cook_income['MeanIncome'].mean()
print(mean_score)

# Step 2
print(cook_income.at[0, 'MeanIncome'] - mean_score)

# Step 3

square_diff = 0
for i in range(cook_income.shape[0]):
    temp_value = (cook_income.at[i, 'MeanIncome'] - mean_score) ** 2
    square_diff += temp_value
    
print(square_diff)

### Step 4 | $w_{ij} = d_{ij}^\alpha$

Let's calculate the distance decay of each pair of locations i and j, if the distance between two places is less than a threshold bandwidth. Suppose we want to formulate the data structure as shown below. 

```python
{i_1: {j_1: distance_decay_1,
       j_2: distance_decay_2,
       j_3: distance_decay_3,
       ...
     }
 ...
 i_87: {j_1: distance_decay_1,
        j_2: distance_decay_2,
        j_3: distance_decay_3,
        ...
       }
}

```

In [ ]:
w_ = {}
dist = 5000
alpha = -1

for i in tqdm(range(cook_income.shape[0])):
    temp_dict = {}
    for j in range(cook_income.shape[0]):
        if i != j:
            temp_dist = cook_income.at[i, 'geometry'].centroid.distance(cook_income.at[j, 'geometry'].centroid)
            if temp_dist <= dist:
                temp_dict[j] = temp_dist ** alpha
                
    w_[i] = temp_dict
    

In [ ]:
# Validation
coords = cook_income.apply(lambda x:x.geometry.centroid.coords[0], axis=1).values

# Calculate weights of each geographical units based on distance decay method. 
w = libpysal.weights.DistanceBand(data=list(coords), # coordinates of each geographical units
                                  threshold=dist, # threshold bandwidth
                                  binary=False, # whether distance decay is employed or not. 
                                  alpha=alpha, # distance decay parameter for weight (default -1.0)
                                  silence_warnings=True
                                 )
# print(w.neighbors[0])
# print(w.weights[0])

print(dict(zip(w.neighbors[0], w.weights[0])))
print("-------")
print(w_[0])

In [ ]:
from copy import deepcopy

In [ ]:
# Calculate the portion of each distance decay value over the entire distance decay value
w__ = deepcopy(w_)

for i in tqdm(w_.keys()):
    for j in w_[i].keys():
        temp_sum = sum(list(w_[i].values()))
        w__[i][j] = w_[i][j] / temp_sum
#         print(i, j, w_[i][j], sum(w_[i].values()), w__[i][j])

In [ ]:
print(w_[0])
print("-------")
print(sum(w_[0].values()))
print("-------")
print(w__[0])
print("-------")
print(sum(w__[0].values()))

In [ ]:
# Reassign the standardized weights of distance decay
w_ = w__

### Step 5 | $W = \sum_{i=1}^{n}\sum_{j=1}^{n} w_{ij}$

In [ ]:
W = 0
for i in range(len(cook_income)):
    for j in w_[i].keys():
#         print(i, j, w_[i][j])
        W += w_[i][j]
        
print(W)

### Step 6 | $\frac{n}{W}$

In [ ]:
n = cook_income.shape[0]

n/W

### Step 7 | $\sum_{i=1}^{n}\sum_{j=1}^{n} w_{ij}(x_i - \overline{x})(x_j - \overline{x})$

In [ ]:
numerator = 0

for i in tqdm(range(cook_income.shape[0])):
    for j in w_[i].keys():
        diff_1 = cook_income.at[i, 'MeanIncome'] - mean_score
        diff_2 = cook_income.at[j, 'MeanIncome'] - mean_score
        
        numerator += w_[i][j] * diff_1 * diff_2
        
print(numerator)

### Step 8: Finalize Moran's I 
$$I = \frac{n}{W} \frac{\sum_{i=1}^{n}\sum_{j=1}^{n} w_{ij}(x_i - \overline{x})(x_j - \overline{x})}{\sum_{i=1}^{n}(x_i - \overline{x})^2}$$

In [ ]:
I = (n/W) * (numerator / square_diff)
I

In [ ]:
# Validation

# Obtain coordinates of each geographical units
coords = cook_income.apply(lambda x:x.geometry.centroid.coords[0], axis=1).values

# Calculate weights of each geographical units based on distance decay method. 
w = libpysal.weights.DistanceBand(data=list(coords), # coordinates of each geographical units
                                  threshold=dist, # threshold bandwidth
                                  binary=False, # whether distance decay is employed or not. 
                                  alpha=alpha, # distance decay parameter for weight (default -1.0)
                                  silence_warnings=True
                                 )
# Focused variable
y = cook_income['MeanIncome']

# Calculate Moran's I
mi = esda.moran.Moran(y, w)

# Print results
print(mi.I) # Moran's I value
print(mi.p_norm) # p-value of the current Moran's I 
print(mi.z_norm) # Z Score of the current Moran's I 

print("---------------")
print(f"Moran's I with {dist} feet radius: {round(mi.I, 3)}, p-value: {round(mi.p_norm, 3)}, z-score: {round(mi.z_norm, 3)}")
print("---------------")

